# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [12]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [21]:
orders=pd.read_csv("../data/Orders.csv")
orders


Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0                0     536365    85123A  2010     12    3     8   
1                1     536365     71053  2010     12    3     8   
2                2     536365    84406B  2010     12    3     8   
3                3     536365    84029G  2010     12    3     8   
4                4     536365    84029E  2010     12    3     8   
...            ...        ...       ...   ...    ...  ...   ...   
397919      541904     581587     22613  2011     12    5    12   
397920      541905     581587     22899  2011     12    5    12   
397921      541906     581587     23254  2011     12    5    12   
397922      541907     581587     23255  2011     12    5    12   
397923      541908     581587     22138  2011     12    5    12   

                                Description  Quantity          InvoiceDate  \
0        white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                       white metal lantern         6  2010-12-01 08:26:00   
2            cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3       knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4            red woolly hottie white heart.         6  2010-12-01 08:26:00   
...                                     ...       ...                  ...   
397919          pack of 20 spaceboy napkins        12  2011-12-09 12:50:00   
397920         children's apron dolly girl          6  2011-12-09 12:50:00   
397921        childrens cutlery dolly girl          4  2011-12-09 12:50:00   
397922      childrens cutlery circus parade         4  2011-12-09 12:50:00   
397923        baking set 9 piece retrospot          3  2011-12-09 12:50:00   

        UnitPrice  CustomerID         Country  amount_spent  
0            2.55       17850  United Kingdom         15.30  
1            3.39       17850  United Kingdom         20.34  
2            2.75       17850  United Kingdom         22.00  
3            3.39       17850  United Kingdom         20.34  
4            3.39       17850  United Kingdom         20.34  
...           ...         ...             ...           ...  
397919       0.85       12680          France         10.20  
397920       2.10       12680          France         12.60  
397921       4.15       12680          France         16.60  
397922       4.15       12680          France         16.60  
397923       4.95       12680          France         14.85  

[397924 rows x 14 columns]

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [53]:
#Total amount spent per customer
customers=orders.groupby("CustomerID").sum().sort_values(by="amount_spent",ascending=False)["amount_spent"].reset_index()
customers

CustomerID  amount_spent
0          14646     280206.02
1          18102     259657.30
2          17450     194550.79
3          16446     168472.50
4          14911     143825.06
...          ...           ...
4334       17956         12.75
4335       16454          6.90
4336       14792          6.20
4337       16738          3.75
4338       13256          0.00

[4339 rows x 2 columns]

In [54]:
#Finding the customers in the top quantiles
top25=customers["amount_spent"].quantile(0.75)

top50=customers["amount_spent"].quantile(0.50)

In [55]:
top25

1661.6400000000003

In [56]:
top50

674.4499999999998

In [68]:
def customerclass(customer):
    if customer["amount_spent"] >=top25:
        return "VIP customer"
    elif customer["amount_spent"] <=top25 and customer["amount_spent"] >=top50:
        return "Important"
    else:
        return "Not a VIP"


In [69]:
customers["customerclass"]=customers.apply(lambda customer: customerclass(customer), axis=1)


In [72]:
customers_class=customers[["CustomerID","customerclass"]]
customers_class.head()

CustomerID customerclass
0       14646  VIP customer
1       18102  VIP customer
2       17450  VIP customer
3       16446  VIP customer
4       14911  VIP customer

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [75]:
orders_class = pd.merge(orders, customers_class, on='CustomerID', how='left')


In [77]:
columns = ['CustomerID','Country','customerclass']
country_customer = pd.DataFrame(orders_class.groupby(columns).groups.keys(), columns=columns)

In [78]:
vipscountry = country_customer[country_customer['customerclass'] == 'VIP customer']

In [79]:
countcustomers = vipscountry[['Country', 'CustomerID']].groupby('Country').count()

In [81]:
countcustomers.nlargest(1,'CustomerID')

CustomerID
Country                   
United Kingdom         932

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [85]:
countryvipsandimportant = country_customer[country_customer["customerclass"].isin(["VIP customer","Important"])]


In [86]:
countcustomers = countryvipsandimportant[['Country', 'CustomerID']].groupby('Country').count()

In [87]:
countcustomers.nlargest(1,'CustomerID')

CustomerID
Country                   
United Kingdom        1908